$\color{ORANGE}{\text{MULTI MODEL IN STEEL   }}$

# Цели и задачи
**Описание ноутбука**

В качестве основы для разбора была взята статья https://dipanshurana.medium.com/steel-defect-detection-image-segmentation-using-keras-and-tensorflow-6118bc586ad2

**Выводы**
- Базовая Unet дала лучший результат. 

**Что еще можно сделать**

# Импортируем модули и данные

In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import matplotlib.patches as patches
import re
import random 
import pickle
import cv2
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [9]:
trainImgPath = "/kaggle/input/severstal-steel-defect-detection/train_images/"
trainCsv = "/kaggle/input/severstal-steel-defect-detection/train.csv"
data=pd.read_csv(trainCsv)
data.ClassId=data.ClassId.astype(int)

train_Img_Id = []
train_class_Id = []
for i in os.listdir(trainImgPath):
    for j in range(1,5):
        train_Img_Id.append(i)
        train_class_Id.append(j)
train_Imgs = pd.DataFrame({'ImageId':train_Img_Id,'ClassId':train_class_Id})
train_Imgs.head(10)
 

In [10]:
data = "/kaggle/input/severstal-steel-defect-detection/train.csv"
train_csv=pd.read_csv(data)


## Подготавливаем данные для обучения

In [11]:
Image_id=[]
label=[]
train_folder_path='/kaggle/input/severstal-steel-defect-detection/train_images/'
for i in os.listdir(train_folder_path): #https://www.geeksforgeeks.org/python-os-listdir-method/
    for j in range(1,5):
        Image_id.append(i)
        label.append(j)

x={'ImageId':Image_id,'ClassId':label}  #https://www.geeksforgeeks.org/creating-a-pandas-dataframe/
train_img=pd.DataFrame(x)
train_img.head(10)

In [12]:
image_size=set()
train_folder_path='/kaggle/input/severstal-steel-defect-detection/train_images/'
for i in os.listdir(train_folder_path): #https://www.geeksforgeeks.org/python-os-listdir-method/
    image_size.add(cv2.imread(train_folder_path+'/'+i).shape)
unique_image=list(image_size)
for x in unique_image:
    print (x)


In [ ]:
train_csv

In [13]:
train_csv.ClassId=train_csv.ClassId.astype(int)
df=pd.merge(train_img,train_csv,how='outer',on=['ImageId','ClassId'])
df.fillna('',inplace=True)
df.head()


In [14]:
#https://www.analyticsvidhya.com/blog/2020/03/pivot-table-pandas-python/
train=pd.pivot_table(df,values='EncodedPixels',index='ImageId',columns='ClassId',aggfunc=np.sum).astype(str)
train=train.reset_index()
train.columns=['image_id','rle_1','rle_2','rle_3','rle_4'] 
train.head()


In [15]:
#Stratified corrosion is a type of corrosion that progresses parallel to the metal surface in such a manner that underlying layers are gradually separated.
#For stratified sampling, we have taken stratified based on minority label priority
#https://economictimes.indiatimes.com/definition/stratified-sampling
defect=[]
stratify=[]
for i in range(len(train)):
    if (train['rle_1'][i] != '' or train['rle_2'][i] != '' or train['rle_3'][i] != '' or train['rle_4'][i] != ''):
        defect.append(1)
    else:
        defect.append(0)
    if train['rle_1'][i] != '':
        stratify.append(1)
    elif train['rle_2'][i] != '':
        stratify.append(2)
    elif train['rle_3'][i] != '':
        stratify.append(3)
    elif train['rle_4'][i] != '':
        stratify.append(4)
    else:
        stratify.append(0)
train['defect']=defect
train['stratify']=stratify

In [ ]:
train.head()


In [16]:
defect_1,defect_2,defect_3,defect_4=[],[],[],[]
for i in range(len(train)):
    if train['rle_1'][i] != '':
        defect_1.append(1)
    else:
        defect_1.append(0)
    if train['rle_2'][i] != '':
        defect_2.append(1)
    else:
        defect_2.append(0)
    if train['rle_3'][i] != '':
        defect_3.append(1)
    else:
        defect_3.append(0)
    if train['rle_4'][i] != '':
        defect_4.append(1)
    else:
        defect_4.append(0)
train['defect_1']=defect_1
train['defect_2']=defect_2
train['defect_3']=defect_3
train['defect_4']=defect_4
train['total_defects']=train['defect_1']+ train['defect_2']+ train['defect_3']+ train['defect_4']
train.head()

In [ ]:
train.shape


In [ ]:
# сохраняем обработанные данные
with open('./data.pkl','wb') as f:
     pickle.dump(train,f)

In [ ]:
with open('./data.pkl','rb') as f:
     train=pickle.load(f)

In [ ]:
train.head()


In [ ]:
train.shape


## Подготовка к обучению

In [ ]:
def f1_score(y_true, y_pred): 
    #https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
    #https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    true_positives=K.sum(K.round(K.clip(y_true*y_pred,0,1)))   #calculates number of true positives
    possible_positives=K.sum(K.round(K.clip(y_true,0,1)))      #calculates number of actual positives
    predicted_positives=K.sum(K.round(K.clip(y_pred,0,1)))     

    #K.epsilon takes care of non-zero divisions
    #was modified by adding the constant epsilon, in order to avoid division by 0. Thus NaN will not be computed.
    precision=true_positives/(predicted_positives +K.epsilon())
    recall=true_positives/(possible_positives+K.epsilon())
    f1_val=2*(precision*recall)/(precision+recall+K.epsilon()) 
    return f1_val

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train,x_test=train_test_split(train,test_size=0.10,stratify=train['stratify'],random_state=0)
x_train,x_val=train_test_split(x_train,test_size=0.20,stratify=x_train['stratify'],random_state=0)
x_train=x_train[['image_id','defect']]
x_val=x_val[['image_id','defect']]
x_test=x_test[['image_id','defect']]
print("x_train {}".format(x_train.shape),"  x_val {}".format(x_val.shape)," x_test {}".format(x_test.shape))

С помощью аугментации данных увеличим число изображений.

In [ ]:
#https://www.analyticsvidhya.com/blog/2020/08/image-augmentation-on-the-fly-using-keras-imagedatagenerator/
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 rotation_range=60)

val_datagen=ImageDataGenerator(rescale=1./255)


train_folder_path='/kaggle/input/severstal-steel-defect-detection/train_images/'

train_image_generator=train_datagen.flow_from_dataframe(dataframe=x_train.astype(str),
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col="defect",
                                                  seed=42,
                                                  shuffle=True,
                                                  batch_size=32,
                                                  class_mode="binary",
                                                  target_size=(256,512))

val_image_generator=val_datagen.flow_from_dataframe(dataframe=x_val.astype(str),
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col="defect",
                                                  batch_size=32,
                                                  seed=42,
                                                  shuffle=True,
                                                  class_mode="binary",
                                                  target_size=(256,512))

# Обучение

## Бинарная классификация

Flattening is No brainer and it simply converts a multi-dimensional object to one-dimensional by re-arranging the elements.

While GlobalAveragePooling is a methodology used for better representation of your vector. It can be 1D/2D/3D.It uses a parser window which moves across the object and pools the data by averaging it (GlobalAveragePooling) or picking max value (GlobalMaxPooling).

Batch normalization, it is a process to make neural networks faster and more stable through adding extra layers in a deep neural network.The new layer performs the standardizing and normalizing operations on the input of a layer coming from a previous layer.A typical neural network is trained using a collected set of input data called batch. Similarly, the normalizing process in batch normalization takes place in batches, not as a single input.

Large neural nets trained on relatively small datasets can overfit the training data.By dropping a unit out, we mean temporarily removing it from the network, along with all its incoming and outgoing connections we can overcome overfitting problem.

Used pre-trained Xception() model by keras without fully-connected layer at the top of the network and later freeze the pre-trained model.

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="severstal", entity="eliseeva")


In [ ]:
wandb.config = {
#   "learning_rate": 0.001,
  "epochs": 20,
  "batch_size": 32
}

In [ ]:
#https://keras.io/api/applications/xception/
base_model=keras.applications.xception.Xception(input_shape=(256,512,3),include_top=False)
base_model.trainable=False

m=base_model.output
# add a global average pooling layer
#https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras 
m=GlobalAveragePooling2D()(m)

# add fully-connected layers
m=Dense(1024,activation='relu')(m)
m=BatchNormalization()(m)   #https://www.analyticsvidhya.com/blog/2021/03/introduction-to-batch-normalization/
m=Dropout(0.3)(m)

#https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/
m=Dense(512,activation='relu')(m) 
m=BatchNormalization()(m)
m= Dropout(0.3)(m)

m=Dense(64,activation='relu')(m)
m=BatchNormalization()(m)
m=Dropout(0.3)(m)

#prediction layer
output=Dense(1,activation='sigmoid')(m)   #Binary Classification thus sigmoid is used 

model=Model(inputs=base_model.input,outputs=output)
model._name="Binary_Classification_Model"
model.summary()

In [ ]:
# log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)

checkpoint_filepath='./binary_Xception_2.h5'
model_checkpoint_callback= keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_f1_score',mode='max',save_best_only=True)
#https://keras.io/api/metrics/ 
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=["accuracy",f1_score])
callback=[model_checkpoint_callback,WandbCallback()]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_image_generator,validation_data=val_image_generator,epochs=20,verbose=1,callbacks=callback)

In [ ]:
model=load_model('./binary_Xception_2.h5',custom_objects={'f1_score':f1_score})


In [ ]:
train_image_generator=val_datagen.flow_from_dataframe(dataframe=x_train.astype(str),
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col="defect",
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="binary",
                                                  target_size=(256,512))


val_image_generator=val_datagen.flow_from_dataframe(dataframe=x_val.astype(str),
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col="defect",
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="binary",
                                                  target_size=(256,512))

test_image_generator=val_datagen.flow_from_dataframe(dataframe=x_test.astype(str),
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col="defect",
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="binary",
                                                  target_size=(256,512))

In [ ]:
print('Training Dataset:\n')
print(model.evaluate(train_image_generator,verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(val_image_generator,verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_image_generator,verbose=1))

Мы видим, что значения метрик и потерь близки для валидационной выборки, тестовой и обучающейся - это значит, что модель не переобучается. f1_score для всего датасета составляет порядка 0.9. В целом модель имеет хорошую производительность.

## Классификация по нескольким меткам

In [ ]:
train=train[train['defect']==1]
x_train,x_test=train_test_split(train,test_size=0.10,stratify=train['stratify'],random_state=0)
x_train,x_val=train_test_split(x_train,test_size=0.20,stratify=x_train['stratify'],random_state=0)
x_train=x_train[['image_id','defect_1','defect_2','defect_3','defect_4']]
x_val=x_val[['image_id','defect_1','defect_2','defect_3','defect_4']]
x_test=x_test[['image_id','defect_1','defect_2','defect_3','defect_4']]
print("x_train {}".format(x_train.shape),"  x_val {}".format(x_val.shape)," x_test {}".format(x_test.shape))

In [ ]:
#https://www.analyticsvidhya.com/blog/2020/08/image-augmentation-on-the-fly-using-keras-imagedatagenerator/
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.1,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 rotation_range=10,
                                 width_shift_range=0.1, 
                                 height_shift_range=0.1)

val_datagen=ImageDataGenerator(rescale=1./255)


train_folder_path='/kaggle/input/severstal-steel-defect-detection/train_images/'
col=["defect_1","defect_2","defect_3","defect_4"]
train_image_generator=train_datagen.flow_from_dataframe(dataframe=x_train,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  class_mode="raw",
                                                  target_size=(256,512))

val_image_generator=val_datagen.flow_from_dataframe(dataframe=x_val,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  class_mode="raw",
                                                  target_size=(256,512))

Далее используем предобученную Xception() без полносвязных слоев в качестве базовой модели в голове?? нейронной сети и с последующей заморозкой предобученной модели.


In [ ]:
#https://keras.io/api/applications/inceptionresnetv2/
base_model=keras.applications.xception.Xception(input_shape=(256,512,3),include_top=False)
base_model.trainable=False

m=base_model.output
# add a global average pooling layer
#https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras 
m=GlobalAveragePooling2D()(m)

# add fully-connected layers
m=Dense(1024,activation='relu')(m)
m=BatchNormalization()(m)   #https://www.analyticsvidhya.com/blog/2021/03/introduction-to-batch-normalization/
m=Dropout(0.4)(m)

#https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/
m=Dense(512,activation='relu')(m) 
m=BatchNormalization()(m)
m= Dropout(0.4)(m)

m=Dense(64,activation='relu')(m)

#prediction layer
output=Dense(4,activation='sigmoid')(m)   

model=Model(inputs=base_model.input,outputs=output)
model._name="Multi_label_Classification_Model"
model.summary()

In [ ]:
checkpoint_filepath='./multi_label.h5'
model_checkpoint_callback=keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_f1_score',mode='max',save_best_only=True)
#https://keras.io/api/metrics/ 
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=["accuracy",f1_score])
callback=[model_checkpoint_callback,WandbCallback()]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_image_generator,validation_data=val_image_generator,epochs=20,verbose=1,callbacks=callback)

In [ ]:
model=load_model('./multi_label.h5',custom_objects={'f1_score':f1_score})


In [ ]:
train_image_generator=val_datagen.flow_from_dataframe(dataframe=x_train,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="raw",
                                                  target_size=(256,512))


val_image_generator=val_datagen.flow_from_dataframe(dataframe=x_val,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="raw",
                                                  target_size=(256,512))

test_image_generator=val_datagen.flow_from_dataframe(dataframe=x_test,
                                                  directory=train_folder_path,
                                                  x_col="image_id",
                                                  y_col=col,
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode="raw",
                                                  target_size=(256,512))

In [ ]:
print('Training Dataset:\n')
print(model.evaluate(train_image_generator,verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(val_image_generator,verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_image_generator,verbose=1))

Для валидационных и тестовых данных потери увеличиваются по сравнению с обучающим сетом, но все же при переходе от валидации к тестированию видим небольшое уменьшение loss, аналогично для f1_score.

## Сегментационная модель

Далее создадим отдельный датасет для каждого вида дефектов.

In [17]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train,x_test=train_test_split(train,test_size=0.10,stratify=train['stratify'],random_state=0)
x_train,x_val=train_test_split(x_train,test_size=0.20,stratify=x_train['stratify'],random_state=0)
print("x_train {}".format(x_train.shape),"  x_val {}".format(x_val.shape)," x_test {}".format(x_test.shape))
print("="*100)

train_1=x_train[x_train['defect_1']==1][['image_id','rle_1']].rename(columns={'image_id':'image_id','rle_1':'rle'})
train_2=x_train[x_train['defect_2']==1][['image_id','rle_2']].rename(columns={'image_id':'image_id','rle_2':'rle'})
train_3=x_train[x_train['defect_3']==1][['image_id','rle_3']].rename(columns={'image_id':'image_id','rle_3':'rle'})
train_4=x_train[x_train['defect_4']==1][['image_id','rle_4']].rename(columns={'image_id':'image_id','rle_4':'rle'})
print("train_1 {}".format(train_1.shape)," train_2 {}".format(train_2.shape)," train_3 {}".format(train_3.shape)," train_4 {}".format(train_4.shape))
print("="*100)

val_1=x_val[x_val['defect_1']==1][['image_id','rle_1']].rename(columns={'image_id':'image_id','rle_1':'rle'})
val_2=x_val[x_val['defect_2']==1][['image_id','rle_2']].rename(columns={'image_id':'image_id','rle_2':'rle'})
val_3=x_val[x_val['defect_3']==1][['image_id','rle_3']].rename(columns={'image_id':'image_id','rle_3':'rle'})
val_4=x_val[x_val['defect_4']==1][['image_id','rle_4']].rename(columns={'image_id':'image_id','rle_4':'rle'})
print("val_1 {}".format(val_1.shape)," val_2 {}".format(val_2.shape)," val_3 {}".format(val_3.shape)," val_4 {}".format(val_4.shape))
print("="*100)

test_1=x_test[x_test['defect_1']==1][['image_id','rle_1']].rename(columns={'image_id':'image_id','rle_1':'rle'})
test_2=x_test[x_test['defect_2']==1][['image_id','rle_2']].rename(columns={'image_id':'image_id','rle_2':'rle'})
test_3=x_test[x_test['defect_3']==1][['image_id','rle_3']].rename(columns={'image_id':'image_id','rle_3':'rle'})
test_4=x_test[x_test['defect_4']==1][['image_id','rle_4']].rename(columns={'image_id':'image_id','rle_4':'rle'})
print("test_1 {}".format(test_1.shape)," test_2 {}".format(test_2.shape)," test_3 {}".format(test_3.shape)," test_4 {}".format(test_4.shape))

In [57]:
import tensorflow as tf


In [7]:
#https://stackoverflow.com/questions/31273652/how-to-calculate-dice-coefficient-for-measuring-accuracy-of-image-segmentation-i
def dice_coef(y_true,y_pred):
    y_true_f=tf.reshape(tf.dtypes.cast(y_true, tf.float32),[-1])
    y_pred_f=tf.reshape(tf.dtypes.cast(y_pred, tf.float32),[-1])
    intersection=tf.reduce_sum(y_true_f*y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)

#https://stackoverflow.com/questions/49785133/keras-dice-coefficient-loss-function-is-negative-and-increasing-with-epochs
def dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return (1-dice_coefficient(y_true, y_pred))

#defining function for calculation of loss function: binary cross entropy + dice loss
def bce_dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))

In [19]:
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def rle_to_mask(rle):
    # CONVERT RLE TO MASK 
    if (pd.isnull(rle))|(rle=='')|(rle=='-1'): 
        return np.zeros((256,800),dtype=np.uint8)  #If the EncodedPixels string is empty an empty mask is returned
    
    height=256
    width=1600
    mask=np.zeros(width*height,dtype=np.uint8)

    array=np.asarray([int(x) for x in rle.split()])
    starts=array[0::2]-1
    lengths=array[1::2]    
    for index,start in enumerate(starts):
        mask[int(start):int(start+lengths[index])]=1
    
    return mask.reshape((height,width),order='F')[::,::2]

The imgaug library provides a very useful feature called Augmentation pipeline. Such a pipeline is a sequence of steps that can be applied in a fixed or random order. This also gives the flexibility to apply certain transformations to a few images and other transformations to other images.

https://www.kaggle.com/parulpandey/overview-of-popular-image-augmentation-packages

In [37]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
Implementing custom data generator
#https://towardsdatascience.com/implementing-custom-data-generators-in-keras-de56f013581c
#https://www.kaggle.com/cdeotte/keras-unet-with-eda
"""


class train_DataGenerator(keras.utils.Sequence):

    def __init__(
        self,
        dataframe,
        batch_size=4,
        shuffle=True,
        preprocess=None,
        info={},
        ):
        self.batch_size = batch_size
        self.df = dataframe
        self.indices = self.df.index.tolist()
        self.preprocess = preprocess
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1)
            * self.batch_size]
        batch = [self.indices[k] for k in index]

        (X, y) = self.__get_data(batch)
        return (X, y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        train_datagen = ImageDataGenerator()

    # https://www.geeksforgeeks.org/python-select-random-value-from-a-list/

        X = np.empty((self.batch_size, 256, 800, 3), dtype=np.float32)  # image place-holders
        Y = np.empty((self.batch_size, 256, 800, 1), dtype=np.int8)  # 1 mask place-holders

        for (i, id) in enumerate(batch):
            X[i] = \
                Image.open('../input/severstal-steel-defect-detection/train_images/'
                            + str(self.df['image_id'
                           ].loc[id])).resize((800, 256))
            Y[i, :, :, 0] = rle_to_mask(self.df['rle'].loc[id])

        t = random.choice([0, 0, 20, 30, 40])
        z = random.choice([0.9, 1])
        flip = random.choice(['True', 'False'])

        param = {
            'flip_horizontal': flip,
            'flip_vertical': flip,
            'tx': t,
            'ty': t,
            'zx': z,
            'zy': z,
            }
        for (i, e) in enumerate(X):
            X[i] = train_datagen.apply_transform(e,
                    transform_parameters=param)
        for (i, f) in enumerate(Y):
            Y[i] = train_datagen.apply_transform(f,
                    transform_parameters=param)

    # preprocess input

        if self.preprocess != None:
            X = self.preprocess(X)

        return (X, Y)


In [31]:
from keras.utils import np_utils


In [35]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# Implementing custom data generator
# https://towardsdatascience.com/implementing-custom-data-generators-in-keras-de56f013581c


class test_DataGenerator(keras.utils.Sequence):

    def __init__(
        self,
        dataframe,
        batch_size=1,
        shuffle=False,
        preprocess=None,
        info={},
        ):
        self.batch_size = batch_size
        self.df = dataframe
        self.indices = self.df.index.tolist()
        self.preprocess = preprocess
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1)
            * self.batch_size]
        batch = [self.indices[k] for k in index]

        (X, y) = self.__get_data(batch)
        return (X, y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        X = np.empty((self.batch_size, 256, 800, 3), dtype=np.float32)  # image place-holders
        Y = np.empty((self.batch_size, 256, 800, 1), dtype=np.int8)  # 1 mask place-holders

        for (i, id) in enumerate(batch):
            X[i] = \
                Image.open('../input/severstal-steel-defect-detection/train_images/'
                            + str(self.df['image_id'
                           ].loc[id])).resize((800, 256))
            Y[i, :, :, 0] = rle_to_mask(self.df['rle'].loc[id])

      # preprocess input

        if self.preprocess != None:
            X = self.preprocess(X)

        return (X, Y)


In [40]:
pip install segmentation-models

In [51]:
%env SM_FRAMEWORK=keras

In [52]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
import imgaug.augmenters as iaa
import segmentation_models as sm
import datetime


In [53]:
#https://segmentation-models.readthedocs.io/en/latest/tutorial.html
#LOAD UNET WITH PRETRAINING FROM IMAGENET
#https://segmentation-models.readthedocs.io/en/latest/tutorial.html
preprocess=get_preprocessing('efficientnetb5')
model=Unet('efficientnetb5',classes=1,activation='sigmoid',encoder_weights='imagenet')
model._name="Segmentation_Model"
model.summary()

### дефект 1

In [59]:
train_batch=train_DataGenerator(train_1,shuffle=True,preprocess=preprocess)    
valid_batch=test_DataGenerator(val_1,preprocess=preprocess)

# log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)

checkpoint_filepath='./segmentation_defect_1.h5'
model_checkpoint_callback=keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/ 
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=25,verbose=1,callbacks=callback)

In [60]:
# segmentation_defect_1.h5
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_1,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_1,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_1,preprocess=preprocess),verbose=1))

For validation,test datasets loss increases and metrics decreases as compared with train data thus better to train model for more epochs (e.g. epoch =100) which might help to reduce loss and increase dice coefficient on unseen data.

In [61]:
def rle2mask(rle):
    # CONVERT RLE TO MASK 
    if (pd.isnull(rle))|(rle=='')|(rle=='-1'): 
        return np.zeros((256,1600) ,dtype=np.uint8)
    
    height= 256
    width = 1600
    mask= np.zeros( width*height ,dtype=np.uint8)

    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]-1
    lengths = array[1::2]    
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
    
    return mask.reshape( (height,width), order='F' )

In [63]:
model=load_model('./segmentation_defect_1.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [67]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


def plot_mask(rle_defect, k, pred):
    train_folder_path = \
        '../input/severstal-steel-defect-detection/train_images/'

  # Create figure and axes

    (fig, ax) = plt.subplots(4, 3, figsize=(14, 9))
    fig.suptitle('Defect_' + str(k) + '_Images', fontsize=20,
                 fontweight='bold')
    for i in range(4):
        image_id = rle_defect[i][0]
        rle = rle_defect[i][1]
        im = Image.open(train_folder_path + str(image_id))
        ax[i, 0].imshow(im)
        ax[i, 0].set_title(image_id)
        mask = rle2mask(rle)
        ax[i, 1].imshow(mask)
        ax[i, 1].set_title('Actual Mask for ' + str(image_id))
        c1 = Image.fromarray(pred[i][:, :, 0])
        ax[i, 2].imshow(np.array(c1.resize((1600, 256))) > 0.5)
        ax[i, 2].set_title('Predicted Mask for ' + str(image_id))
    fig.set_facecolor('tan')
    plt.show()


train dataset

In [65]:
train_preds=model.predict_generator(test_DataGenerator(train_1[10:14],preprocess=preprocess),verbose=1)

In [68]:
plot_mask(train_1[10:14].values,1,train_preds)

Validation Dataset

In [69]:
val_preds=model.predict_generator(test_DataGenerator(val_1[10:14],preprocess=preprocess),verbose=1)

In [70]:
plot_mask(val_1[10:14].values,1,val_preds)

Test Dataset

In [71]:
test_preds=model.predict_generator(test_DataGenerator(test_1[10:14],preprocess=preprocess),verbose=1)

In [72]:
plot_mask(test_1[10:14].values,1,test_preds)

### дефект 2

In [73]:
train_batch=train_DataGenerator(train_2,shuffle=True,preprocess=preprocess)    
valid_batch=test_DataGenerator(val_2,preprocess=preprocess)

# log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)

checkpoint_filepath='./segmentation_defect_2.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/ 
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=25,verbose=1,callbacks=callback)

In [74]:
model=load_model('./segmentation_defect_2.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [75]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_2,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_2,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_2,preprocess=preprocess),verbose=1))

One can observe the value of loss and metrics are similar for train,validation and test datasets thus the model is not over-fitting. On unseen data (test) dice coefficient increases and loss decreases as compared to train, validation datasets thus, model is performing really good.

trianing dataset

In [76]:
train_preds=model.predict_generator(test_DataGenerator(train_2[10:14],preprocess=preprocess),verbose=1)

In [77]:
plot_mask(train_2[10:14].values,2,train_preds)

валиация

In [78]:
val_preds=model.predict_generator(test_DataGenerator(val_2[10:14],preprocess=preprocess),verbose=1)

In [79]:
plot_mask(val_2[10:14].values,2,val_preds)

тест

In [80]:
test_preds=model.predict_generator(test_DataGenerator(test_2[10:14],preprocess=preprocess),verbose=1)

In [81]:
plot_mask(test_2[10:14].values,2,test_preds)

### дефект 3

In [83]:
train_batch=train_DataGenerator(train_3,shuffle=True,preprocess=preprocess)    
valid_batch=test_DataGenerator(val_3,preprocess=preprocess)

# log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)

checkpoint_filepath='./segmentation_defect_3.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/ 
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=6,verbose=1,callbacks=callback)

In [84]:
model=load_model('./segmentation_defect_3.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [85]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_3,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_3,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_3,preprocess=preprocess),verbose=1))

One can observe the value of loss and metrics are similar for train,validation and test datasets thus the model is not over-fitting. On unseen data (test) dice coefficient slightly increases and loss decreases as compared to train, validation datasets thus, model is performing really good.

training dataset

In [86]:
train_preds=model.predict_generator(test_DataGenerator(train_3[12:16],preprocess=preprocess),verbose=1)

In [87]:
plot_mask(train_3[12:16].values,3,train_preds)

Validation Dataset

In [88]:
val_preds=model.predict_generator(test_DataGenerator(val_3[12:16],preprocess=preprocess),verbose=1)

In [89]:
plot_mask(val_3[12:16].values,3,val_preds)

### дефект 4

In [90]:
train_batch=train_DataGenerator(train_4,shuffle=True,preprocess=preprocess)    
valid_batch=test_DataGenerator(val_4,preprocess=preprocess)

log_dir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True,write_grads=True)

checkpoint_filepath='./segmentation_defect_4.h5'
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_dice_coef',mode='max',save_best_only=True)
#https://keras.io/api/metrics/ 
#https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(optimizer='adam',loss=bce_dice_loss,metrics=[dice_coef])
callback=[model_checkpoint_callback]
#https://datascience.stackexchange.com/questions/34444/what-is-the-difference-between-fit-and-fit-generator-in-keras
history=model.fit_generator(train_batch,validation_data=valid_batch,epochs=25,verbose=1,callbacks=callback)

In [91]:
model=load_model('./segmentation_defect_4.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [92]:
print('Training Dataset:\n')
print(model.evaluate(test_DataGenerator(train_4,preprocess=preprocess),verbose=1))
print("="*100)
print('\nValidation Dataset:\n')
print(model.evaluate(test_DataGenerator(val_4,preprocess=preprocess),verbose=1))
print("="*100)
print('\nTest Dataset:\n')
print(model.evaluate(test_DataGenerator(test_4,preprocess=preprocess),verbose=1))

For validation,test datasets loss slightly increases and metrics slightly decreases as compared with train data but overall values are similar thus model is not over-fitting. For better results on unseen data train model for more epochs (e.g. epoch =100) based on the available resources (memory).

trainig dataset

In [93]:
train_preds=model.predict_generator(test_DataGenerator(train_4[14:18],preprocess=preprocess),verbose=1)

In [94]:
plot_mask(train_4[14:18].values,4,train_preds)

Validation Dataset

In [95]:
val_preds=model.predict_generator(test_DataGenerator(val_4[14:18],preprocess=preprocess),verbose=1)

In [96]:
plot_mask(val_4[14:18].values,4,val_preds)

Test Dataset

In [97]:
test_preds=model.predict_generator(test_DataGenerator(test_4[14:18],preprocess=preprocess),verbose=1)

In [98]:
plot_mask(test_4[14:18].values,4,test_preds)

# Final Pipeline


In [2]:
def f1_score(y_true, y_pred): 
    #https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
    #https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    true_positives=K.sum(K.round(K.clip(y_true*y_pred,0,1)))   #calculates number of true positives
    possible_positives=K.sum(K.round(K.clip(y_true,0,1)))      #calculates number of actual positives
    predicted_positives=K.sum(K.round(K.clip(y_pred,0,1)))     

    #K.epsilon takes care of non-zero divisions
    #was modified by adding the constant epsilon, in order to avoid division by 0. Thus NaN will not be computed.
    precision=true_positives/(predicted_positives +K.epsilon())
    recall=true_positives/(possible_positives+K.epsilon())
    f1_val=2*(precision*recall)/(precision+recall+K.epsilon()) 
    return f1_val

In [3]:
test_image=[i for i in os.listdir('../input/severstal-steel-defect-detection/test_images')]


In [4]:
len(test_image)


In [5]:
def sum_pixel(i):
    return sum([int(k) for k in i.split(' ')[1::2]])

In [1]:
binary=load_model('../input/models/binary_Xception_2.h5',custom_objects={'f1_score':f1_score})
multi=load_model('../input/models/multi_label.h5',custom_objects={'f1_score':f1_score})
segment_1=load_model('../input/models/segmentation_defect_1.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
segment_2=load_model('../input/models/segmentation_defect_2.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
segment_3=load_model('../input/models/segmentation_defect_3.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
segment_4=load_model('../input/models/segmentation_defect_4.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [108]:
test_folder_path='../input/severstal-steel-defect-detection/test_images'


In [109]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


def threshold(X):
    t = []

    for i in range(len(X)):

        if sum_pixel(X['rle_1'].iloc[i]) >= 300 and sum_pixel(X['rle_1'
                ].iloc[i]) <= 13500 and X['defect'].iloc[i] >= 0.4 \
            and X['defect_1'].iloc[i] >= 0.5:
            t.append([X['image_id'].iloc[i] + '_1', X['rle_1'].iloc[i]])
        else:
            t.append([X['image_id'].iloc[i] + '_1', ''])

        if sum_pixel(X['rle_2'].iloc[i]) >= 500 and sum_pixel(X['rle_2'
                ].iloc[i]) <= 9000 and X['defect'].iloc[i] >= 0.4 \
            and X['defect_2'].iloc[i] >= 0.5:
            t.append([X['image_id'].iloc[i] + '_2', X['rle_2'].iloc[i]])
        else:
            t.append([X['image_id'].iloc[i] + '_2', ''])

        if sum_pixel(X['rle_3'].iloc[i]) >= 900 and sum_pixel(X['rle_3'
                ].iloc[i]) <= 140000 and X['defect'].iloc[i] >= 0.5 \
            and X['defect_3'].iloc[i] >= 0.6:
            t.append([X['image_id'].iloc[i] + '_3', X['rle_3'].iloc[i]])
        else:
            t.append([X['image_id'].iloc[i] + '_3', ''])

        if sum_pixel(X['rle_4'].iloc[i]) >= 2400 and sum_pixel(X['rle_4'
                ].iloc[i]) <= 120000 and X['defect'].iloc[i] >= 0.4 \
            and X['defect_4'].iloc[i] >= 0.5:
            t.append([X['image_id'].iloc[i] + '_4', X['rle_4'].iloc[i]])
        else:
            t.append([X['image_id'].iloc[i] + '_4', ''])

    df = pd.DataFrame(t, columns=['imageid_classid', 'rle'])
    return df


Depending on mask area(plot drawn in EDA) and class probability thresholds are decided for each defects respectively.

In [110]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


def predict(X):

    preprocess = get_preprocessing('efficientnetb5')
    datagen = ImageDataGenerator(rescale=1. / 255)
    data_generator = datagen.flow_from_dataframe(
        dataframe=X,
        directory=test_folder_path,
        x_col='image_id',
        class_mode=None,
        target_size=(256, 512),
        batch_size=1,
        shuffle=False,
        )
    a = []

    pred_binary = binary.predict_generator(data_generator)
    pred_multi = multi.predict_generator(data_generator)
    classify = pd.DataFrame(pred_multi, columns=['defect_1', 'defect_2'
                            , 'defect_3', 'defect_4'])
    classify['defect'] = pred_binary
    classify['image_id'] = X['image_id']

    batch = test_DataGenerator(X, preprocess=preprocess)
    pred_1 = segment_1.predict_generator(batch)
    pred_2 = segment_2.predict_generator(batch)
    pred_3 = segment_3.predict_generator(batch)
    pred_4 = segment_4.predict_generator(batch)
    for i in range(len(pred_1)):
        v1 = mask2rle(np.array(Image.fromarray(pred_1[i][:, :, 0]
                      >= 0.5).resize((1600, 256))).astype(int))
        v2 = mask2rle(np.array(Image.fromarray(pred_2[i][:, :, 0]
                      >= 0.5).resize((1600, 256))).astype(int))
        v3 = mask2rle(np.array(Image.fromarray(pred_3[i][:, :, 0]
                      >= 0.5).resize((1600, 256))).astype(int))
        v4 = mask2rle(np.array(Image.fromarray(pred_4[i][:, :, 0]
                      >= 0.5).resize((1600, 256))).astype(int))
        a.append([X.image_id.iloc[i], v1, v2, v3, v4])
    segment = pd.DataFrame(a, columns=['image_id', 'rle_1', 'rle_2',
                           'rle_3', 'rle_4'])

    df = classify.merge(segment, on=['image_id'])

    df1 = threshold(df)
    return df1


First compute binary and multi-label classification later use segmentation model and after checking thresholds for each defect finally predicts Encoded pixels for each image id and class id .


In [125]:
test=pd.DataFrame(test_image,columns=['image_id'])
test1=test[:1]
test1=test1.reset_index().drop('index',axis=1)

In [126]:
test1

In [128]:
data=predict()


In [111]:
test=pd.DataFrame(test_image,columns=['image_id'])
test1=test[0:20]
test1=test1.reset_index().drop('index',axis=1)
data=predict(test1)
data

In [112]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
a = []
b = []
for i in data.imageid_classid.values:
    (k, l) = i.split('_')
    a.append(k)
    b.append(l)


In [113]:
df=pd.DataFrame(columns=['image_Id','class_Id','EncodedPixels'])
df['image_id']=a
df['class_id']=b
df['EncodedPixels']=data.rle.values
df.drop(['image_Id','class_Id'],axis=1,inplace=True)

In [ ]:
#https://www.analyticsvidhya.com/blog/2020/03/pivot-table-pandas-python/
df1=pd.pivot_table(df,values='EncodedPixels',index='image_id',columns='class_id',aggfunc=np.sum).astype(str)
df1=df1.reset_index()
df1.columns=['image_id','rle_1','rle_2','rle_3','rle_4'] 
df1